In [17]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import matplotlib.pyplot as plt
import os
import tqdm

import seaborn as sns
from torchvision.io import read_image
import torchvision.transforms as T
from torchvision.utils import make_grid
from attrdict import AttrDict
import torch
import yaml
from sklearn.model_selection import StratifiedKFold
import copy
import pickle
# from tqdm import tqdm_notebook

# additional lightning 

import pytorch_lightning as pl
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning import callbacks
from pytorch_lightning.callbacks.progress import ProgressBarBase
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import LightningDataModule, LightningModule


# pytorch
import torch
from torch import nn
import torch.nn.functional as F
from timm import create_model


from utility import *

In [19]:
import pandas as pd

train_df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')

In [21]:
test_transformation = T.Compose([
                T.Resize([224,224]),# imgnet needs at least 224
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225] ), # imgnet requirements 
                ]
            )

# Load weights and inference

https://pytorch-lightning.readthedocs.io/en/latest/common/weights_loading.html

In [24]:
import torchvision
from torchvision import models

criterion = torch.nn.BCEWithLogitsLoss()

i=0
#model = pawNetBasic.load_from_checkpoint(checkpoint_path=f"../input/pawnet-ans/paw_greater0_pseudo-labeling_pawpularity__version_{i}_checkpoints_best_loss.ckpt",criterion=criterion,model_config=model_config)
model = models.resnet18(pretrained=True)

#model = model.to("cuda")
model = model.eval()

img_path = '../input/petfinder-pawpularity-score/train/0007de18844b0dbbb5e1f607da0606e0.jpg'
with open(img_path, 'rb') as f:
    img = Image.open(f)
    img = (img).convert("RGB")
    img = T.ToTensor()(img)
    f.close()

input = test_transformation(img)

input = input.unsqueeze(0)

output = model(input)
output = F.softmax(output, dim=1)
prediction_score, pred_label_idx = torch.topk(output, 1)

pred_label_idx.squeeze_(), prediction_score.squeeze().item()
#predicted_label = idx_to_labels[str(pred_label_idx.item())][1]
#print('Predicted:', predicted_label, '(', prediction_score.squeeze().item(), ')')

In [33]:
"""saliency maps"""
!pip install captum 

from captum.attr import IntegratedGradients
from captum.attr import GradientShap
from captum.attr import Saliency
from captum.attr import NoiseTunnel
from captum.attr import visualization as viz

import torch
import torch.nn.functional as F

from PIL import Image

import os
import json
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

from captum.attr import Saliency

def attribute_image_features(algorithm, input, **kwargs):
    net.zero_grad()
    tensor_attributions = algorithm.attribute(input,
                                              target=labels[ind],
                                              **kwargs
                                             )
    
    return tensor_attributions

saliency = Saliency(model)
grads = saliency.attribute(input, target=0)
grads = np.transpose(grads.squeeze().cpu().detach().numpy(), (1, 2, 0))


_ = viz.visualize_image_attr(None, np.transpose(input.squeeze().cpu().detach().numpy(), (1,2,0)), 
                      method="original_image", title="Original Image")

_ = viz.visualize_image_attr(grads, np.transpose(input.squeeze().cpu().detach().numpy(), (1,2,0)), method="blended_heat_map", sign="absolute_value",
                          show_colorbar=True, title="Overlayed Gradient Magnitudes")

In [52]:
from captum.attr import LayerGradCam
import skimage.transform

layer_gc = LayerGradCam(model, model.layer4[1].conv2)

# get for each class 
attr_1 = layer_gc.attribute(input, 1, relu_attributions=True)
pred = model(input)

plt.imshow(np.transpose(input.squeeze().cpu().detach().numpy(), (1,2,0)))
plt.imshow(skimage.transform.resize(attr_1.detach().cpu().numpy()[0][0], (224,224)), alpha=0.5, cmap='jet')
plt.axis("off")

In [30]:
_ = viz.visualize_image_attr(None, np.transpose(input.squeeze().cpu().detach().numpy(), (1,2,0)),
                      method="original_image", title="Original Image")

In [5]:
train_df.iloc[0].Pawpularity

In [ ]:
!pip install captum 

from captum.attr import IntegratedGradients
from captum.attr import GradientShap
from captum.attr import Saliency
from captum.attr import NoiseTunnel
from captum.attr import visualization as viz

import torch
import torch.nn.functional as F

from PIL import Image

import os
import json
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

# Create IntegratedGradients object and get attributes
integrated_gradients = IntegratedGradients(model)
attributions_ig = integrated_gradients.attribute(input, target=pred_label_idx, n_steps=200)

# create custom colormap for visualizing the result
default_cmap = LinearSegmentedColormap.from_list('custom blue', 
                                                 [(0, '#ffffff'),
                                                  (0.25, '#000000'),
                                                  (1, '#000000')], N=256)


# visualize the results using the visualize_image_attr helper method
_ = viz.visualize_image_attr_multiple(np.transpose(attributions_ig.squeeze().cpu().detach().numpy(), (1,2,0)),
                             np.transpose(img.squeeze().cpu().detach().numpy(), (1,2,0)),
                             methods=["original_image", "heat_map"],
                             signs=['all', 'positive'],
                             cmap=default_cmap,
                             show_colorbar=True)